# Lecture 2 - Designing

In [39]:
# Let's import the required libraries
import sqlite3
import pandas as pd

# Connect to the database file 
# (it will create the file if it doesn't exist)
conn = sqlite3.connect('mbta.db')
# MBTA stands for Massachusetts Bay Transportation Authority

## CREATE TABLE

In [2]:
conn.execute("""
CREATE TABLE "riders" (
    "id",
    "name"
);
""")

conn.execute("""
CREATE TABLE "stations" (
    "id",
    "name",
    "line"
);
""")

conn.execute("""
CREATE TABLE "visits" (
    "rider_id",
    "station_id"
);
""")

# Commit changes to save the table in the database
conn.commit()


In [3]:
pd.read_sql_query(
    """
    SELECT *
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)

,type,name,tbl_name,rootpage,sql
0,table,riders,riders,2,"CREATE TABLE ""riders"" (\n ""id"",\n ""name""\n)"
1,table,stations,stations,3,"CREATE TABLE ""stations"" (\n ""id"",\n ""nam..."
2,table,visits,visits,4,"CREATE TABLE ""visits"" (\n ""rider_id"",\n ..."


## DROP TABLE - Delete Table

In [7]:
# Drop the table 'riders' if it exists
conn.execute('DROP TABLE IF EXISTS riders;')  
# IF EXISTS ensures no error if the table doesn't exist

# Drop the table 'stations' and 'visits' if it exists
conn.execute('DROP TABLE IF EXISTS stations;')  
conn.execute('DROP TABLE IF EXISTS visits;')  

# Commit changes
conn.commit()

## Data Types and Storage Classes

SQLite has five storage classes:
- NULL: nothing, or empty value
- INTEGER: numbers without decimal points
- REAL: decimal or floating point numbers
- TEXT: characters or strings
- BLOB: Binary Large Object, for storing objects in binary (useful for images, audio etc.)

A storage class can hold several data types.For example, these are the data types of the INTEGER storage class:

- INTEGER storage class
    Data Types:
    - 0-byte integer
    - 1-byte integer
    - 2-byte integer
    - 3-byte integer
    - 4-byte integer
    - 6-byte integer
    - 8-byte integer

SQLite takes care of storing the input value under the right data type.

## Type Affinities

It is possible to specify the data type of a column. Columns in SQLite are said to have TYPE AFFINITIES, meaning that they try to convert an input value into the type they have an affinity for.

For example: If we try to insert an integer 25 into a column with a type affinity for text, it will be converted into an text data type.

## Adding Types to our Tables

Inside the file schema.sql, let’s type out the schemas again, but with the affinity types this time.

In [8]:
conn.execute("""
CREATE TABLE riders (
    "id" INTEGER,
    "name" TEXT
);
""")

conn.execute("""
CREATE TABLE stations (
    "id" INTEGER,
    "name" TEXT,
    "line" TEXT
);
""")

conn.execute("""
CREATE TABLE visits (
    "rider_id" INTEGER,
    "station_id" INTEGER
);
""")

# Commit changes to save the table in the database
conn.commit()

In [9]:
pd.read_sql_query(
    """
    SELECT *
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)

,type,name,tbl_name,rootpage,sql
0,table,riders,riders,2,"CREATE TABLE riders (\n ""id"" INTEGER,\n ..."
1,table,stations,stations,4,"CREATE TABLE stations (\n ""id"" INTEGER,\n ..."
2,table,visits,visits,3,"CREATE TABLE visits (\n ""rider_id"" INTEGER,..."


If we don’t specify a type affinity of a column in SQLite, what happens?

- The default type affinity is numeric, so the column would get assigned the numeric type affinity.

## Table Constraints - PRIMARY KEY and FOREIGN KEY

We can use table constraints to impose restrictions on certain values in our tables.

A primary key column must have unique values. The table constraint we use for this is PRIMARY KEY.

A constraint on a foreign key value is that it must be found in the primary key column of the related table! This table constraint is called, predictably, FOREIGN KEY.

Let’s add primary and foreign key constraints to our schema.sql file.

In [19]:
# Drop the table 'riders','stations' and 'visits' if it exists
conn.execute('DROP TABLE IF EXISTS riders;')  
conn.execute('DROP TABLE IF EXISTS stations;')  
conn.execute('DROP TABLE IF EXISTS visits;')  
# IF EXISTS ensures no error if the table doesn't exist

conn.execute("""
CREATE TABLE riders (
    "id" INTEGER,
    "name" TEXT,
    PRIMARY KEY("id")
);
""")

conn.execute("""
CREATE TABLE stations (
    "id" INTEGER,
    "name" TEXT,
    "line" TEXT,
    PRIMARY KEY("id")
);
""")

conn.execute("""
CREATE TABLE visits (
    "rider_id" INTEGER,
    "station_id" INTEGER,
    FOREIGN KEY("rider_id") REFERENCES "riders"("id"),
    FOREIGN KEY("station_id") REFERENCES "stations"("id")
);
""")

# Commit changes to save the table in the database
conn.commit()

pd.read_sql_query(
    """
    SELECT "name","rootpage","sql"
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)

,name,rootpage,sql
0,riders,3,"CREATE TABLE riders (\n ""id"" INTEGER,\n ..."
1,stations,4,"CREATE TABLE stations (\n ""id"" INTEGER,\n ..."
2,visits,2,"CREATE TABLE visits (\n ""rider_id"" INTEGER,..."


## Column Constraints

A column constraint is a type of constraint that applies to a specified column in the table.

SQLite has four column constraints:
- CHECK: allows checking for a condition, like all values must be greater than 0
- DEFAULT: uses a default value if none is supplied for a row
- NOT NULL: dictates that a null or empty value cannot be inserted into the column
- UNIQUE: dictates that every value in this column must be unique

In [47]:
# Drop the table 'riders','stations' and 'visits' if it exists
conn.execute('DROP TABLE IF EXISTS riders;')  
conn.execute('DROP TABLE IF EXISTS stations;')  
conn.execute('DROP TABLE IF EXISTS visits;')  
# IF EXISTS ensures no error if the table doesn't exist

conn.execute("""
CREATE TABLE riders (
    "id" INTEGER,
    "name" TEXT,
    PRIMARY KEY("id")
);
""")

conn.execute("""
CREATE TABLE stations (
    "id" INTEGER,
    "name" TEXT NOT NULL UNIQUE,
    "line" TEXT NOT NULL,
    PRIMARY KEY("id")
);
""")

conn.execute("""
CREATE TABLE visits (
    "rider_id" INTEGER,
    "station_id" INTEGER,
    FOREIGN KEY("rider_id") REFERENCES "riders"("id"),
    FOREIGN KEY("station_id") REFERENCES "stations"("id")
);
""")

# Commit changes to save the table in the database
conn.commit()

pd.read_sql_query(
    """
    SELECT "name","rootpage","sql"
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)

,name,rootpage,sql
0,riders,3,"CREATE TABLE riders (\n ""id"" INTEGER,\n ..."
1,stations,4,"CREATE TABLE stations (\n ""id"" INTEGER,\n ..."
2,visits,6,"CREATE TABLE visits (\n ""rider_id"" INTEGER,..."


Primary key columns and by extension, foreign key columns must always have unique values, so there is no need to explicitly specify the NOT NULL or UNIQUE column constraints

In [56]:
pd.read_sql_query(
    "PRAGMA table_info(stations);",
    conn
)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,1,None,0
2,2,line,TEXT,1,None,0


## Altering Tables

ALTER TABLE ... RENAME TO ... :
ALTER TABLE ... ADD COLUMN ...
ALTER TABLE ... RENAME COLUMN ... TO ...
ALTER TABLE ... DROP COLUMN ...


In [46]:
conn.execute('DROP TABLE IF EXISTS swipes;')
conn.commit()

In [48]:
conn.execute('DROP TABLE IF EXISTS riders;')

df=pd.read_sql_query(
    """
    SELECT "sql"
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)
print(df["sql"][0])
print(df["sql"][1])

CREATE TABLE stations (
    "id" INTEGER,
    "name" TEXT NOT NULL UNIQUE,
    "line" TEXT NOT NULL,
    PRIMARY KEY("id")
)
CREATE TABLE visits (
    "rider_id" INTEGER,
    "station_id" INTEGER,
    FOREIGN KEY("rider_id") REFERENCES "riders"("id"),
    FOREIGN KEY("station_id") REFERENCES "stations"("id")
)


ALTER TABLE ... RENAME TO ... :

In [49]:
conn.execute("""
ALTER TABLE visits
RENAME TO swipes;
""")

df=pd.read_sql_query(
    """
    SELECT *
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)
df

,type,name,tbl_name,rootpage,sql
0,table,stations,stations,4,"CREATE TABLE stations (\n ""id"" INTEGER,\n ..."
1,table,swipes,swipes,6,"CREATE TABLE ""swipes"" (\n ""rider_id"" INTEGE..."


ALTER TABLE ... RENAME TO ... :
ADD COLUMN ...

In [54]:
conn.execute("""
ALTER TABLE "swipes" ADD COLUMN "type" INTEGER;
""")

In [55]:
df=pd.read_sql_query(
    """
    SELECT "sql"
    FROM sqlite_master
    WHERE type = 'table';
    """,
    conn
)
print(df["sql"][0])
print(df["sql"][1])

CREATE TABLE stations (
    "id" INTEGER,
    "name" TEXT NOT NULL UNIQUE,
    "line" TEXT NOT NULL,
    PRIMARY KEY("id")
)
CREATE TABLE "swipes" (
    "rider_id" INTEGER,
    "station_id" INTEGER, "type" INTEGER,
    FOREIGN KEY("rider_id") REFERENCES "riders"("id"),
    FOREIGN KEY("station_id") REFERENCES "stations"("id")
)


In [53]:
conn.execute("""
ALTER TABLE "swipes" DROP COLUMN "type";
""")